In [27]:
import sys
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sqlalchemy import create_engine

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

import nltk
nltk.download(['punkt', 'wordnet'])

from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from joblib import dump

from plotly.graph_objs import Bar
import plotly.graph_objs as go
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
df = pd.read_csv(r'data\disaster_merged_data.csv')
df.columns

C:\Users\User\AppData\Local\Temp\ipykernel_23868\774445859.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'data\disaster_merged_data.csv')


Index(['Unnamed: 0', 'id', 'message', 'original', 'genre', 'related',
       'request', 'offer', 'aid_related', 'medical_help', 'medical_products',
       'search_and_rescue', 'security', 'military', 'child_alone', 'water',
       'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees',
       'death', 'other_aid', 'infrastructure_related', 'transport',
       'buildings', 'electricity', 'tools', 'hospitals', 'shops',
       'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
       'storm', 'fire', 'earthquake', 'cold', 'other_weather',
       'direct_report'],
      dtype='object')

In [10]:
label_names=['other_infrastructure', 'food', 'security', 'shelter', 'refugees', 'earthquake', 
              'medical_products', 'missing_people', 'death', 'shops', 'related', 'offer', 
              'medical_help', 'hospitals', 'aid_centers', 'weather_related', 'cold', 'request', 
              'storm', 'water', 'floods', 'military', 'other_aid', 'aid_related', 'direct_report', 
              'fire', 'clothing', 'money', 'buildings', 'transport', 'tools', 'search_and_rescue', 
              'other_weather', 'child_alone', 'infrastructure_related', 'electricity']

label_counts = [df[x].sum() for x in label_names]

graphs = [
        {
            'data': [
                Bar(
                    x=label_names,
                    y=label_counts
                )
            ],

            'layout': {
                'title': 'Distribution of Label',
                'yaxis': {
                    'title': "Count"
                },
                'xaxis': {
                    'title': "Label"
                }
            }
        }
    ]

In [24]:
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')

def tokenize(text):
    # Tokenize the text
    tokens = word_tokenize(text)
    # Initialize the WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()

    # List to hold cleaned tokens
    clean_tokens = []
    for tok in tokens:
        # Lemmatize, convert to lower case and strip white spaces
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

# Initialize CountVectorizer with the tokenize function as the tokenizer
CV = CountVectorizer(tokenizer=tokenize)
# Initialize TfidfTransformer
TT = TfidfTransformer()

# Some example text data
text_data = df['message']

# Fit and transform the text data with CountVectorizer
count_matrix = CV.fit_transform(text_data)

# Use TfidfTransformer to transform the count matrix to a tf-idf representation
tfidf_matrix = TT.fit_transform(count_matrix)

# Now you can use tfidf_matrix as input for a machine learning model.
# Create a DataFrame with the TF-IDF features
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=CV.get_feature_names_out())



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
c:\Users\User\anaconda3\envs\udacity2\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [40]:
# Sum up the TF-IDF scores of each word
word_scores = tfidf_df.sum(axis=0).to_dict()

# Normalize the TF-IDF scores to get font sizes between 10 and 200
max_font_size = 200
min_font_size = 10
max_score = max(word_scores.values())
min_score = min(word_scores.values())

# Normalizing function
def normalize(score):
    return min_font_size + (score - min_score) / (max_score - min_score) * (max_font_size - min_font_size)

# Create the scatter plot
fig = go.Figure()

for word, score in word_scores.items():
    fig.add_trace(go.Scatter(
        x=[np.random.rand()],
        y=[np.random.rand()],
        text=word,
        mode='text',
        textfont=dict(size=normalize(score)),
        showlegend=False
    ))

# Set the layout to have no axes or grid
fig.update_layout(
    xaxis={'showgrid': False, 'showticklabels': False, 'zeroline': False},
    yaxis={'showgrid': False, 'showticklabels': False, 'zeroline': False}
)

# Show the figure
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Load your data
# Assuming 'df' is your DataFrame and the last column is the target variable
X = tfidf_df  # all rows, all columns except the last
y = df['food']   # all rows, just the last column

# Standardize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# PCA transformation
pca = PCA(n_components=2)  # Reduce to 2 dimensions for visualization
X_pca = pca.fit_transform(X_scaled)


In [51]:
y = y[X_pca[:,0] < 500]
y = y[X_pca[:,1] < 500]
X_pca = X_pca[X_pca[:,0] < 500]
X_pca = X_pca[X_pca[:,1] < 500]

# Create a scatter plot
plt.figure(figsize=(8, 6))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='viridis', edgecolor='k', s=50, alpha=0.6)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('PCA of Dataset')
plt.colorbar(scatter)
plt.show()


IndexError: Boolean index has wrong length: 26216 instead of 26215